In [1]:
from data_downloader import download_example
from data_analyzer import DataAnalyzer
from datasist.structdata import detect_outliers
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)


In [2]:
df = download_example()


File with 'international' in its name already exists.


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
def df_to_table(df):
    data = [df.columns.tolist()] + df.values.tolist()
    table = Table(data)
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))
    return table


In [ ]:
from io import BytesIO
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Spacer
from reportlab.lib import colors
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from datasist.structdata import detect_outliers  # You need to have datasist library installed for this

# Define the DataAnalyzer class (paste the class definition you provided)

# Create a sample DataFrame
data = {
    'Column1': [10, 15, 20, 25, 30, 35],
    'Column2': [2.5, 3.7, 4.8, 6.2, 7.1, 100],  # Introduce an outlier here
    'Column3': ['A', 'B', 'C', 'D', 'E', 'F']
}
df = pd.DataFrame(data)

# Initialize the DataAnalyzer with the DataFrame
data_analyzer = DataAnalyzer(df)

# Run the methods on the DataFrame
duplicate_percentage, null_percentage = data_analyzer.duplicates_nulls_percentage(df)
outliers_info = data_analyzer.remove_outliers(df)
summary_dict = data_analyzer.dataframe_summary_to_dict(df)
data_analyzer.encode_scale_features(df)

# Convert null_percentage Series to string before formatting
null_percentage_str = null_percentage.apply(lambda x: f'{x:.2f}%')

In [ ]:
import pandas as pd
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph

class DataAnalyzer:
    def duplicates_nulls_percentage(self, df):
        duplicate_percentage = (df.duplicated().mean() * 100).round(2)
        null_percentage = (df.isnull().mean() * 100).round(2)
        return duplicate_percentage, null_percentage

    def remove_outliers(self, df):
        # Mocked outlier info for demonstration
        outliers_info = [
            {'Name': 'column1', 'Percentage': 0.5, 'Number_Of_Outliers': 10},
            {'Name': 'column2', 'Percentage': 0.2, 'Number_Of_Outliers': 5}
        ]
        return outliers_info

def generate_pdf_report(df, output_path, analyzer):
    try:
        doc = SimpleDocTemplate(output_path, pagesize=letter)
        elements = []

        styles = getSampleStyleSheet()
        title_style = styles['Title']
        header_style = styles['Heading1']
        body_style = styles['BodyText']

        duplicate_percentage, null_percentage = analyzer.duplicates_nulls_percentage(df)
        outliers_info = analyzer.remove_outliers(df)

        # Data description section
        data_description = df.describe(include='all').to_html(classes='table')

        elements.append(Paragraph("<b>Data Description:</b>", title_style))
        elements.append(Paragraph(data_description, body_style))

        elements.append(Paragraph("<b>Duplicates and Nulls Analysis:</b>", title_style))
        elements.append(Paragraph(f"Duplicates Percentage: {duplicate_percentage}%", header_style))
        elements.append(Paragraph(f"Nulls Percentage: {null_percentage}%", header_style))

        elements.append(Paragraph("<b>Outliers Analysis:</b>", title_style))
        for info in outliers_info:
            column_name = info['Name']
            percentage = info['Percentage']
            num_outliers = info['Number_Of_Outliers']
            elements.append(Paragraph(f"Column: {column_name}", header_style))
            elements.append(Paragraph(f"Outliers Percentage: {percentage * 100:.2f}%", header_style))
            elements.append(Paragraph(f"Number of Outliers: {num_outliers}", header_style))

        doc.build(elements)
        print(f"PDF report generated and saved to {output_path}")
    except Exception as e:
        print("Error occurred while generating PDF report.")
        print(f"Error message: {str(e)}")

# Example usage
if __name__ == "__main__":
    data = {'column1': [1, 2, 2, 3, 3],
            'column2': [10, 15, 18, 25, 30]}
    df = pd.DataFrame(data)
    
    output_path = "report.pdf"
    
    analyzer = DataAnalyzer()
    generate_pdf_report(df, output_path, analyzer)


In [3]:
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Spacer, Paragraph
import pandas as pd
from data_downloader import download_example
from data_analyzer import DataAnalyzer
from datasist.structdata import detect_outliers
pd.set_option('display.float_format', lambda x: '%.2f' % x)

def generate_pdf_report(df, output_path, numerical_columns):
    try:
        doc = SimpleDocTemplate(output_path, pagesize=letter)
        elements = []

        styles = getSampleStyleSheet()
        title_style = styles['Title']
        header_style = styles['Heading1']
        body_style = styles['BodyText']

        for column_name in numerical_columns:
            # Add title for column description
            elements.append(Paragraph(f"<b>Description of {column_name}</b>", title_style))

            # Convert column data to a DataFrame for description
            column_description = df[column_name].describe().to_frame()

            # Convert DataFrame to HTML table
            column_description_html = column_description.to_html(classes='table')

            # Add the HTML table to the report
            elements.append(Paragraph(column_description_html, body_style))

            elements.append(Spacer(1, 12))  # Add space

        doc.build(elements)
        print(f"PDF report generated and saved to {output_path}")
    except Exception as e:
        print("Error occurred while generating PDF report.")
        print(f"Error message: {str(e)}")

In [ ]:
data = {'column1': [1, 2, 2, 3, 3],
            'column2': [10, 15, 18, 25, 30]}
df = pd.DataFrame(data)

output_path = "report.pdf"

analyzer = DataAnalyzer(df)
generate_pdf_report(df, output_path, analyzer.numerical_columns)

In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from datasist.structdata import detect_outliers
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph
from reportlab.lib.styles import getSampleStyleSheet
from data_downloader import download_example
from data_analyzer import DataAnalyzer
from datasist.structdata import detect_outliers
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Your DataAnalyzer class here...

class ReportGenerator:
    def __init__(self, output_file):
        self.output_file = output_file
        self.data = []

    def add_description(self, text):
        self.data.append(Paragraph(text, getSampleStyleSheet()["Normal"]))
        self.data.append(Table([("",)]))  # Add an empty table as a separator

    def add_table(self, headers, rows):
        table_data = [headers] + rows
        table = Table(table_data)
        table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
            ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ]))
        self.data.append(table)
        self.data.append(Table([("----------",)]))  # Add an empty table as a separator

    def generate_pdf(self):
        doc = SimpleDocTemplate(self.output_file, pagesize=letter)
        doc.build(self.data)

# Example usage
if __name__ == "__main__":
    df = download_example() # Replace with your data source
    analyzer = DataAnalyzer(df)

    report = ReportGenerator("output.pdf")

    # Description for numerical, categorical, and datetime columns
    report.add_description("Column Types:")
    numerical_columns = ", ".join(analyzer.numerical_columns)
    categorical_columns = ", ".join(analyzer.categorical_columns)
    datetime_columns = ", ".join(analyzer.datetime_columns)
    report.add_table(["Type", "Column Names"],
                     [("Numerical Columns", numerical_columns),
                      ("Categorical Columns", categorical_columns),
                      ("Datetime Columns", datetime_columns)])

    # Duplicate Percentage table
    report.add_description("Duplicate Percentage:")
    duplicate_percentage, _ = analyzer.duplicates_nulls_percentage(df)
    report.add_table(["Metric", "Percentage"], [("Duplicate Percentage", f"{duplicate_percentage}%")])

    # Null Percentage table
    report.add_description("Null Percentage:")
    _, null_percentage = analyzer.duplicates_nulls_percentage(df)
    null_table_data = [["Column Name", "Percentage"]]
    for column_name, percentage in null_percentage.items():
        null_table_data.append([column_name, f"{percentage:.2f}%"])
    report.add_table(null_table_data[0], null_table_data[1:])

    # Description for outliers
    report.add_description("Outliers:")
    outliers_info = analyzer.remove_outliers(df)
    report.add_table(["Column Name", "Percentage of Outliers", "Number of Outliers"],
                     [(info['Name'], f"{info['Percentage']*100:.2f}%", info['Number_Of_Outliers']) for info in outliers_info])

    # More descriptions and tables can be added for other analysis methods

    # Generate the PDF
    report.generate_pdf()

File with 'international' in its name already exists.
